# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [20]:
# Do all imports and installs here
import pandas as pd
import findspark
import gzip
import json
import dataExtractUtils as de
import uuid

findspark.init()

from pyspark.sql import SparkSession

#Locate spark on local machine

print(dir(de))

print(pd.get_option("display.max_colwidth"))
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_rows",None)
print(pd.get_option("display.max_rows"))

['NA', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'constants', 'datetime', 'matchCountriesInAffiliation', 'processArticle', 'processAuthor', 'processPublishedDate', 'uuid']
None
60


### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

In [ ]:
#Setup SparkSession
spark = SparkSession.builder.getOrCreate()

In [2]:
# Read in the data here
#Test file path /home/jswainston/Downloads/0.json.gz
#df = spark.read.json("/home/jswainston/Downloads/April2022CrossrefPublicDataFile/0.json.gz",multiLine=True)
#df.printSchema()
#df.show()


with gzip.open("/home/jswainston/Downloads/April2022CrossrefPublicDataFile/0.json.gz", 'r') as fin:        # 4. gzip
    json_bytes = fin.read()                      # 3. bytes (i.e. UTF-8)

json_str = json_bytes.decode('utf-8')            # 2. string (i.e. JSON)
data = json.loads(json_str)                      # 1. data
print("data object type is " + str(type(data)))

print("Data has the following keys " + str(data.keys()))
print("The type of the value corresponding to the item key is " + str(type(data["items"])))
print("The length of the list contained in the items dict is " + str(len(data["items"])))
print(type(data["items"][0]))
print(len((data["items"][0])))
print(data["items"][0].keys())
print(data["items"][0].keys())
print("date part type test " + str(type(data["items"][0]["published"]["date-parts"])))
print(type(data["items"][0]["published"]["date-parts"][0]))
print(type(data["items"][0]["published"]["date-parts"][0][0]))
print(data["items"][0]["published"]["date-parts"][0][0])

#access author affiliations
#for item in data["items"]:
#    if "author" in item:
#        for author in item["author"]:
#            for affiliation in author["affiliation"]:
#                print(affiliation["name"]) 





data object type is <class 'dict'>
Data has the following keys dict_keys(['items'])
The type of the value corresponding to the item key is <class 'list'>
The length of the list contained in the items dict is 5000
<class 'dict'>
33
dict_keys(['URL', 'resource', 'member', 'score', 'created', 'license', 'ISSN', 'container-title', 'issued', 'issue', 'prefix', 'reference-count', 'indexed', 'author', 'DOI', 'is-referenced-by-count', 'published', 'published-print', 'subject', 'published-online', 'content-domain', 'title', 'link', 'source', 'type', 'publisher', 'journal-issue', 'volume', 'references-count', 'issn-type', 'deposited', 'page', 'short-container-title'])
dict_keys(['URL', 'resource', 'member', 'score', 'created', 'license', 'ISSN', 'container-title', 'issued', 'issue', 'prefix', 'reference-count', 'indexed', 'author', 'DOI', 'is-referenced-by-count', 'published', 'published-print', 'subject', 'published-online', 'content-domain', 'title', 'link', 'source', 'type', 'publisher', 'jou

In [6]:
# test retreiving country values from affiliations
i=0
for item in data["items"]:
    print("DOI " + item["DOI"])
    if "author" in item:
        for author in item["author"]:
            i = i + 1
            print("Author number " + str(i))
            print(uuid.uuid4())
            for affiliation in author["affiliation"]:
                affiliationCountries = de.matchCountriesInAffiliation(affiliation["name"])
                print(affiliationCountries) 

DOI 10.1149/1.1392467
Author number 1
09ee0c64-7236-4aaa-b418-0fbdda56e5fe
['Italy']
Author number 2
4f0a72bb-367a-4e55-99a0-4a9d117a2ef6
['Italy']
Author number 3
7e7ce492-1dfd-4e4d-aad4-2501c0066cbc
['Italy']
Author number 4
5aa2e72c-0070-4f05-ba3c-632546c6aad8
['Italy']
DOI 10.1108/09564239810199969
Author number 5
4f061231-47e8-435f-9f89-f449f5d43a33
DOI 10.1108/08880450010310462
Author number 6
687b5b8b-eac6-43dd-bbd7-53b3e51a8816
DOI 10.1134/1.1493384
Author number 7
af0e9052-02d1-422e-b913-79a9c648a439
Author number 8
22a60ee9-7732-436c-ae0b-b3bf90239d98
Author number 9
e55c4879-e624-4a0d-bd59-e69ad3bc6f28
DOI 10.1128/cdli.9.4.753-762.2002
Author number 10
aaf43f16-3fa6-47d4-b29e-ede94d6c5692
['France']
DOI 10.1108/01437729710169300
Author number 11
bac3974d-0021-4fe3-8dc3-5a09ae749f0f
DOI 10.1016/s8755-4615(97)90024-4
Author number 12
d725aca3-e988-47e1-a886-2c867dfd18c2
DOI 10.1108/09578230010378377
Author number 13
bb418a1e-fb63-4e0c-a17a-069fa57ea07e
Author number 14
359d9a5

In [5]:
#Extract data into DataFrames for each entity we are inerested in

articleData = {
    'article_id' : [],
    'DOI' : [],
    'title' : [],
    'published_date' : []
}

authorData = {
    'author_id' : [],
    'article_id' : [],
    'first_name' : [],
    'last_name' : []
}

affiliationData = {
    'affiliation_id' : [],
    'author_id' : [],
    'country_name' : []
}

affiliationCountry = {
    'country_ID' : [],
    'affiliation_id' : []
}

affiliationErrors = {
    'affiliation_id' : [],
    'author_id' : [],
    'affiliation_name' : []
}

for item in data["items"]:
    if "type" in item:
        if item["type"] == 'journal-article':
            article = de.processArticle(item)
            articleData['article_id'].append(article[0])
            articleData['DOI'].append(article[1])
            articleData['title'].append(article[2])
            articleData['published_date'].append(article[3])

            authors = de.processAuthor(item,article[0]) # pass article id in so it can be used as foreign key for author
            affiliationErrorData = authors[2]
            authorAffiliations = authors[1]
            authors = authors[0]

            
            for author in authors:
                authorData['author_id'].append(author[0])
                authorData['article_id'].append(author[1])
                authorData['first_name'].append(author[2])
                authorData['last_name'].append(author[3])

            if authorAffiliations is not None:
                for affiliation in authorAffiliations:
                    affiliationData['affiliation_id'].append(affiliation[0])
                    affiliationData['author_id'].append(affiliation[1])
                    affiliationData['country_name'].append(affiliation[2])

            if affiliationErrorData is not None:
                for affiliationError in affiliationErrorData:
                    affiliationErrors['affiliation_id'].append(affiliationError[0])
                    affiliationErrors['author_id'].append(affiliationError[1])
                    affiliationErrors['affiliation_name'].append(affiliationError[2])

            

articles = pd.DataFrame(articleData)
authors = pd.DataFrame(authorData)
affiliations = pd.DataFrame(affiliationData)
affiliationErrors = pd.DataFrame(affiliationErrors)



In [4]:
# show articles sample
articles.head()

,article_id,DOI,title,published_date
0,197f055d-abaf-4391-90dc-4d38a28da160,10.1149/1.1392467,[A Kinetic Model for the Metallorganic Chemica...,1999-09-01
1,1a4e6fea-25d0-46dd-b5f4-52f9d2cd91b4,10.1108/09564239810199969,[Services marketing management competencies: a...,1998-03-01
2,ed3c09f4-417b-4e6b-8f83-4afad52a3279,10.1108/08880450010310462,[Statistics and teaching in departments of lib...,2000-03-01
3,f5b970db-9f1d-41eb-a588-a791f01ada53,10.1134/1.1493384,[Quasicrystals with the infinite point group a...,2002-06-01
4,c806c8fb-578f-409d-b4f5-4dd23844558d,10.1128/cdli.9.4.753-762.2002,[Evaluation of Events Occurring at Mucosal Sur...,2002-07-01


In [5]:
# show authors sample
authors.head()

,author_id,article_id,first_name,last_name
0,55443a81-71aa-4e2f-8300-c1484871cdf7,197f055d-abaf-4391-90dc-4d38a28da160,Carlo,Cavallotti
1,e87ab4d3-bfdb-4996-8b06-ad87ba932b10,197f055d-abaf-4391-90dc-4d38a28da160,Valeria,Bertani
2,142ee9ff-8dc8-486c-8963-7b69544ee796,197f055d-abaf-4391-90dc-4d38a28da160,Maurizio,Masi
3,c6962572-8f70-4a0e-a662-58e8a6703640,197f055d-abaf-4391-90dc-4d38a28da160,Sergio,Carrà
4,91a9f6f1-5a3c-4a43-b573-ed132d5d9a9c,1a4e6fea-25d0-46dd-b5f4-52f9d2cd91b4,Audrey,Gilmore


In [6]:
# show affiliation sample
affiliations.head()

,affiliation_id,author_id,country_name
0,ba2e0ef7-6479-44b3-91b7-bee1e22551e5,55443a81-71aa-4e2f-8300-c1484871cdf7,Italy
1,c56abd3a-2a37-4faf-bac6-755dcce41399,e87ab4d3-bfdb-4996-8b06-ad87ba932b10,Italy
2,2d1d1f01-89d0-4441-ad06-d8afeea5076e,142ee9ff-8dc8-486c-8963-7b69544ee796,Italy
3,3dc84878-e6c4-4085-8a9a-7fc44acba4ac,c6962572-8f70-4a0e-a662-58e8a6703640,Italy
4,1a2a3ce0-d92c-4122-b9fb-3dc46df2b67c,c358f482-a9d5-4c24-8b70-0f6f9973fbd9,France


In [18]:
#show affiliations where a country wasn't found
affiliationErrors["affiliation_name"]

0                                                 Department of Chemistry, University of California, Irvine, California 92697-2025
1                                                 Department of Chemistry, University of California, Irvine, California 92697-2025
2                                                 Department of Chemistry, University of California, Irvine, California 92697-2025
3              Bioprocess and Formulation Technology, Pharmacia, 700 Chesterfield Parkway North, St. Louis, Missouri 63198, U.S.A.
4              Bioprocess and Formulation Technology, Pharmacia, 700 Chesterfield Parkway North, St. Louis, Missouri 63198, U.S.A.
                                                                   ...                                                            
1078                 From the Cardiovascular Imaging Center, Cardiovascular Division, University of Virginia, Charlottesville, Va.
1079    Department of Electrical Engineering and Computer Sciences, University of C

In [ ]:




df_spark = spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')

df_spark.head()

### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

In [ ]:
# Performing cleaning tasks here





### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [ ]:
# Write code here

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.